In [1]:
# Libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation, Dropout, Lambda, Cropping2D
from keras.layers.pooling import MaxPool2D
import sklearn
import cv2
import os
import csv
import matplotlib as plt
from sklearn.model_selection import train_test_split
from PIL import Image
import PIL

/Users/tunc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Global Parameters and Swithes
flip = True
flipSuffixForImage = '_flipped.jpg'
flipSuffixForCSV = '_flipped.csv'

In [3]:
# Image Flipper
imagePathRoot = "./TrainingData"
allFileNames = [x for x in os.listdir(imagePathRoot) if  not x.startswith(".")]

if flip:
    for fileName in allFileNames:
        print("Generating Flipped Images for",fileName)
        singleSamplePath = imagePathRoot +'/'+ fileName
        flippedImagePath = singleSamplePath + '/Flipped_IMG'

        if not os.path.exists(flippedImagePath):
            os.makedirs(flippedImagePath)

        imagePath = singleSamplePath + '/IMG'

        allImageNames = os.listdir(imagePath)
        for imgName in allImageNames:
            image = Image.open(imagePath + '/' + imgName)
            flipImage = image.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            flipImage.save(flippedImagePath+'/'+imgName.replace('.jpg','')+flipSuffixForImage, "JPEG")

    print("Images are flipped and saved successfully..")
else:
    print("Image flipping is not necessary anymore")

Generating Flipped Images for Sample_1
Generating Flipped Images for Sample_2
Generating Flipped Images for Sample_3
Generating Flipped Images for Sample_4
Generating Flipped Images for Sample_5
Generating Flipped Images for Sample_6
Generating Flipped Images for Sample_7
Images are flipped and saved successfully..


In [4]:
#Log Data Flipper
if flip:
    for fileName in allFileNames:
        singleDrivingLogPath = imagePathRoot +'/'+ fileName +'/'+ "driving_log.csv"
        allLinesFlipped = []
        originalWithOutPaths = []
        with open(singleDrivingLogPath) as csvfile:
            reader = csv.reader(csvfile)
            for line in reader:
                orgLine = line.copy()
                for i in range(0,3): # First three columns are document names and suffix has to be added
                    #root = '/'.join(str(path) for path in line[i].split("/")[:-1])
                    line[i] = line[i].split("/")[-1].split(".")[0] + flipSuffixForImage
                    orgLine[i] = orgLine[i].split("/")[-1].split(".")[0] + ".jpg" 
                steeringAng = line[3] # Here steering is multiplied by -1
                line[3] = str(-1 * float(steeringAng))
                allLinesFlipped.append(line)
                originalWithOutPaths.append(orgLine)
        with open(imagePathRoot +'/'+ fileName +'/'+ "driving_log" + flipSuffixForCSV, "w") as csv_file:
            writer = csv.writer(csv_file,delimiter=',')
            for line in allLinesFlipped:
                writer.writerow(line)
        with open(imagePathRoot +'/'+ fileName +'/'+ "driving_log" + "_without_path.csv", "w") as csv_file:
            writer = csv.writer(csv_file,delimiter=',')
            for line in originalWithOutPaths:
                writer.writerow(line)                
    print("Csv Files are flipped and saved successfully..")
else:
    print("Csv Flipping is not necessary anymore")

Csv Files are flipped and saved successfully..


In [5]:
#Image Reader - Random Examples from each Sample

STOP
imageDim = '''TODO-(depth,height,width)''' 

NameError: name 'STOP' is not defined

In [ ]:
# Show how much of the Image has to be cropped
top = '''TODO-integer'''
bottom = '''TODO-integer'''
left = '''TODO-integer'''
right = '''TODO-integer'''

croppedImageDim = (imageDim[0],imageDim[1]-(top+bottom),imageDim[2]-(left+right))





In [ ]:
# NVIDIA Architecture + My improvements

def myNet(train_Gen, validation_Gen, numOfTrainSample, numOfValSample, ep):

    #Create the Keras Model
    model = Sequential()

    #Crop Images 
    model.add(Cropping2D(cropping=((top,bottom),(left,right)),input_shape=imageDim))

    #Normalization and Mean center
    model.add(Lambda(lambda x: (x/255.0)-0.5)) # TODO maybe also output_shape=croppedImageDim OR No size Paramenters at all?

    #Convolutional Layer 1
    model.add(Convolution2D(24,5,5))
    model.add(Activation('relu'))

    #Convolutional Layer 2
    model.add(Convolution2D(36,5,5))
    model.add(Activation('relu'))

    #Convolutional Layer 3
    model.add(Convolution2D(48,5,5))
    model.add(Activation('relu'))

    #Convolutional Layer 4
    model.add(Convolution2D(64,3,3))
    model.add(Activation('relu'))

    #Convolutional Layer 5
    model.add(Convolution2D(64,3,3))
    model.add(Activation('relu'))

    #Flatten
    model.add(Flatten())

    #Fully Connected Layer 1
    model.add(Dense(100))
    model.add(Dropout(0.5))
    model.add(Activation('relu'))

    #Fully Connected Layer 2
    model.add(Dense(50))
    model.add(Dropout(0.5)) #TODO ? MAX Pooling?

    #Fully Connected Layer 3
    model.add(Dense(10))
    model.add(Activation('relu'))

    #Final Output
    model.add(Dense(1))

    model.compile(loss='mse',optimizer='adam')
    historyObj = model.fit_generator(X_Train_Gen, samples_per_epoch=numOfTrainSample,
                        validation_data=validation_Gen, nb_val_samples=numOfValSample
                        nb_epoch = ep, verbose=1)

    return historyObj # Can be used for Visualing historyObj.history['loss'] and historyObj.history['loss'] && also check out Keys: print(history_object.history.keys())
